import pandas as pd

In [1]:
import pandas as pd

In [2]:
meteo_2003_2024 = pd.read_csv("meteo_2003_2024.csv")
meteo_2003_2024.head()

,Data,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media
0,2003-01-01,5.4,1123.000000,21.091667,26.0,18.8,18.250000,84.500000,10.1,2.395833
1,2003-01-02,0.6,1652.214286,22.233333,27.2,18.9,18.033333,78.583333,10.9,2.741667
2,2003-01-03,8.4,1365.285714,21.658333,26.6,18.3,18.266667,81.708333,11.6,3.562500
3,2003-01-04,25.4,1423.785714,21.350000,26.8,17.5,17.716667,81.250000,13.9,4.216667
4,2003-01-05,0.8,1469.846154,21.879167,27.1,17.4,17.162500,75.875000,10.8,4.137500


In [8]:
meteo_2003_2024.shape

(8036, 10)

In [5]:
burn_area_2003_2024 = pd.read_csv(r"C:\Users\sofia\Documents\Data_Analytics_Ironhack\Projects\google_brain\Code\focos_2003_2024.csv")
burn_area_2003_2024.head()

,id_bdq,foco_id,lat,lon,data_pas,pais,estado,municipio,bioma,data,hora
0,8699076,546e7062-5abb-11e8-911c-28924ad12c5c,-15.807,-47.474,2003-02-10 16:53:00,Brasil,DISTRITO FEDERAL,BRASÍLIA,Cerrado,2003-02-10,16:53:00
1,10039136,5e8b5f03-5abb-11e8-911c-28924ad12c5c,-15.896,-48.017,2003-05-17 16:53:00,Brasil,DISTRITO FEDERAL,BRASÍLIA,Cerrado,2003-05-17,16:53:00
2,10266418,23ad605e-5abb-11e8-911c-28924ad12c5c,-15.989,-47.769,2003-06-02 16:53:00,Brasil,DISTRITO FEDERAL,BRASÍLIA,Cerrado,2003-06-02,16:53:00
3,10266417,23ad605d-5abb-11e8-911c-28924ad12c5c,-15.824,-48.026,2003-06-02 16:53:00,Brasil,DISTRITO FEDERAL,BRASÍLIA,Cerrado,2003-06-02,16:53:00
4,10317857,5ee086ae-5abb-11e8-911c-28924ad12c5c,-15.654,-48.216,2003-06-07 17:11:00,Brasil,DISTRITO FEDERAL,BRASÍLIA,Cerrado,2003-06-07,17:11:00


In [7]:
burn_area_2003_2024.shape

(5052, 11)

In [54]:
import pandas as pd

# === 1) Garantir que as datas estão no formato certo ===
burn_area_2003_2024["data"] = pd.to_datetime(burn_area_2003_2024["data"], errors="coerce").dt.normalize()
meteo_2003_2024["Data"] = pd.to_datetime(meteo_2003_2024["Data"], errors="coerce").dt.normalize()

# === 2) Filtrar pelas coordenadas de Brasília ===
lat0, lon0 = -15.7939, -47.8828
dlat, dlon = 0.5, 0.5
mask_bb = (
    burn_area_2003_2024["lat"].between(lat0 - dlat, lat0 + dlat)
    & burn_area_2003_2024["lon"].between(lon0 - dlon, lon0 + dlon)
)
focos_bb = burn_area_2003_2024[mask_bb].copy()

# === 3) Agregar por dia ===
focos_dia = (
    focos_bb.groupby("data")
            .size()
            .reset_index(name="n_focos")
            .sort_values("data")
)

# Garantir intervalo completo de 2003-01-01 a 2024-12-31
inicio = pd.to_datetime("2003-01-01")
fim = pd.to_datetime("2024-12-31")
idx = pd.date_range(inicio, fim, freq="D")

focos_dia = (
    focos_dia.set_index("data")
             .reindex(idx)
             .fillna({"n_focos": 0})
             .rename_axis("data")
             .reset_index()
)

focos_dia["n_focos"] = focos_dia["n_focos"].astype(int)
focos_dia["target"] = (focos_dia["n_focos"] > 0).astype(int)

# === 4) Merge com meteorologia diária ===
df_final_2003_2024 = focos_dia.merge(
    meteo_2003_2024, 
    left_on="data", 
    right_on="Data", 
    how="left"
).drop(columns=["Data"])

# === 5) Features históricas (sem leakage) ===
df_final_2003_2024 = df_final_2003_2024.sort_values("data").reset_index(drop=True)

df_final_2003_2024["focos_ultimos_3d"] = (
    df_final_2003_2024["n_focos"].rolling(3, min_periods=1).sum().shift(1)
)
df_final_2003_2024["focos_ultimos_7d"] = (
    df_final_2003_2024["n_focos"].rolling(7, min_periods=1).sum().shift(1)
)

if "precipitacao_mm" in df_final_2003_2024.columns:
    df_final_2003_2024["chuva_7d"] = (
        df_final_2003_2024["precipitacao_mm"].rolling(7, min_periods=1).sum().shift(1)
    )
if "temp_media" in df_final_2003_2024.columns:
    df_final_2003_2024["temp_7d"] = (
        df_final_2003_2024["temp_media"].rolling(7, min_periods=1).mean().shift(1)
    )
if "humidade_media" in df_final_2003_2024.columns:
    df_final_2003_2024["humidade_7d"] = (
        df_final_2003_2024["humidade_media"].rolling(7, min_periods=1).mean().shift(1)
    )

# === 6) Variáveis temporais ===
s = pd.to_datetime(df_final_2003_2024["data"])
df_final_2003_2024["mes"] = s.dt.month
df_final_2003_2024["dia_semana"] = s.dt.weekday

# === 7) Salvar CSV ===
df_final_2003_2024.to_csv("df_brasilia_focos_meteo_2003_2024.csv", index=False)

print(df_final_2003_2024.head(), df_final_2003_2024.shape)


        data  n_focos  target  precipitacao_mm  radiacao_media  temp_media  \
0 2003-01-01        0       0              5.4     1123.000000   21.091667   
1 2003-01-02        0       0              0.6     1652.214286   22.233333   
2 2003-01-03        0       0              8.4     1365.285714   21.658333   
3 2003-01-04        0       0             25.4     1423.785714   21.350000   
4 2003-01-05        0       0              0.8     1469.846154   21.879167   

   temp_max  temp_min  temp_orvalho_media  humidade_media  vento_rajada_max  \
0      26.0      18.8           18.250000       84.500000              10.1   
1      27.2      18.9           18.033333       78.583333              10.9   
2      26.6      18.3           18.266667       81.708333              11.6   
3      26.8      17.5           17.716667       81.250000              13.9   
4      27.1      17.4           17.162500       75.875000              10.8   

   vento_vel_media  focos_ultimos_3d  focos_ultimos_7d  

In [55]:
df_final_2003_2024.tail()

,data,n_focos,target,precipitacao_mm,radiacao_media,temp_media,temp_max,temp_min,temp_orvalho_media,humidade_media,vento_rajada_max,vento_vel_media,focos_ultimos_3d,focos_ultimos_7d,chuva_7d,temp_7d,humidade_7d,mes,dia_semana
8031,2024-12-27,0,0,7.8,1551.814286,21.504167,27.3,17.1,18.004167,81.791667,8.2,2.395833,0.0,0.0,79.0,21.327381,85.255952,12,4
8032,2024-12-28,0,0,10.4,1126.785714,22.120833,27.7,18.3,17.483333,77.125000,6.6,1.600000,0.0,0.0,80.8,21.156548,86.172619,12,5
8033,2024-12-29,0,0,19.0,866.421429,20.891667,27.6,17.1,17.245833,81.666667,5.2,1.729167,0.0,0.0,88.0,21.164286,85.339286,12,6
8034,2024-12-30,0,0,2.0,1093.800000,21.108696,26.7,18.0,17.921739,83.434783,12.6,1.969565,0.0,0.0,86.4,21.269643,84.113095,12,0
8035,2024-12-31,0,0,0.0,1233.692857,21.175000,26.7,18.0,17.187500,78.750000,12.1,2.020833,0.0,0.0,81.2,21.221480,83.943064,12,1


In [66]:
# === Teste rápido para validar se não está tudo a zero ===
dias_com_foco = (df_final_2003_2024["n_focos"] > 0).sum()
total_dias = df_final_2003_2024.shape[0]
perc_dias_com_foco = dias_com_foco / total_dias * 100

print(f"Dias com pelo menos 1 foco: {dias_com_foco} ({perc_dias_com_foco:.2f}%)")
print(f"Dias sem foco: {total_dias - dias_com_foco}")


Dias com pelo menos 1 foco: 1328 (16.53%)
Dias sem foco: 6708


**KNN Regression**

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

In [56]:
features = [
    'precipitacao_mm', 'temp_media', 'humidade_media', 'chuva_7d', 'temp_7d', 'humidade_7d',
    'focos_ultimos_3d', 'focos_ultimos_7d', 'mes', 'dia_semana'
]

X = df_final_2003_2004[features].fillna(0)
y = df_final_2003_2004["n_focos"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [57]:
# simple model
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [58]:
#predicting
y_pred = knn.predict(X_test)

In [59]:
# metrics
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"R²: {r2:.4f}, RMSE: {rmse:.4f}")

R²: 0.1383, RMSE: 2.0800


**KNN Classifier**

In [60]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [61]:
# 1) Features and target
features = [
    'precipitacao_mm', 'temp_media', 'humidade_media',
    'chuva_7d', 'temp_7d', 'humidade_7d',
    'focos_ultimos_3d', 'focos_ultimos_7d',
    'mes', 'dia_semana'
]

X = df_final_2003_2004[features].fillna(0)
y = df_final_2003_2004['target']

In [62]:
# 2) Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [63]:
# 3) KNN Model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [64]:
# 4) Predections
y_pred = knn.predict(X_test)

In [65]:
# 5) Evaluation
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[1233  107]
 [ 152  116]]
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      1340
           1       0.52      0.43      0.47       268

    accuracy                           0.84      1608
   macro avg       0.71      0.68      0.69      1608
weighted avg       0.83      0.84      0.83      1608



**Logistic Regression**

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

# === 1) Escolher features e target
features = [
    "precipitacao_mm", "radiacao_media", "temp_media", "temp_max", "temp_min",
    "temp_orvalho_media", "humidade_media", "vento_rajada_max", "vento_vel_media",
    "focos_ultimos_3d", "focos_ultimos_7d", "chuva_7d", "temp_7d", "humidade_7d",
    "mes", "dia_semana"
]
target = "target"

# Remover linhas com NaN nas features
df_ml = df_final_2003_2024.dropna(subset=features + [target]).copy()

X = df_ml[features]
y = df_ml[target]

# === 2) Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# === 3) Criar e treinar modelo
model = LogisticRegression(max_iter=1000, class_weight="balanced", solver="liblinear")
model.fit(X_train, y_train)

# === 4) Previsões e avaliação
y_pred = model.predict(X_test)

print("=== Matriz de confusão ===")
print(confusion_matrix(y_test, y_pred))
print("\n=== Relatório de classificação ===")
print(classification_report(y_test, y_pred, digits=3))


=== Matriz de confusão ===
[[1075  255]
 [  36  226]]

=== Relatório de classificação ===
              precision    recall  f1-score   support

           0      0.968     0.808     0.881      1330
           1      0.470     0.863     0.608       262

    accuracy                          0.817      1592
   macro avg      0.719     0.835     0.745      1592
weighted avg      0.886     0.817     0.836      1592

